# Plant Pathology Hypertuning

### Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

# import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit
import copy

import utils

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [2]:
print_every = 100
assert(torch.cuda.is_available())
gpu_dtype = torch.cuda.FloatTensor

In [3]:
imgs_tr = utils.getXSamples(100)

In [4]:
# class ChunkSampler(sampler.Sampler):
#     """Samples elements sequentially from some offset. 
#     Arguments:
#         num_samples: # of desired datapoints
#         start: offset where we should start selecting from
#     """
#     def __init__(self, num_samples, start = 0):
#         self.num_samples = num_samples
#         self.start = start

#     def __iter__(self):
#         return iter(range(self.start, self.start + self.num_samples))

#     def __len__(self):
#         return self.num_samples

# NUM_TRAIN = len(X_tr) #49000
# NUM_VAL = 1000

# cifar10_train = dset.CIFAR10('./cs175/datasets', train=True, download=True,
#                            transform=T.ToTensor())
# loader_tr = DataLoader(imgs_tr, batch_size=64,sampler=ChunkSampler(NUM_TRAIN, 0))

# cifar10_val = dset.CIFAR10('./cs175/datasets', train=True, download=True,
#                            transform=T.ToTensor())
# loader_val = DataLoader(cifar10_val, batch_size=64, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

# cifar10_test = dset.CIFAR10('./cs175/datasets', train=False, download=True,
#                           transform=T.ToTensor())
# loader_test = DataLoader(cifar10_test, batch_size=64)


### Hypertuning

In [5]:
# # This sets the model in "training" mode. This is relevant for some layers that may have different behavior
# # in training mode vs testing mode, such as Dropout and BatchNorm. 
# fixed_model_gpu.train()

# # Load one batch at a time.
# # for t, (x, y) in enumerate(loader_train):
#     x_var = Variable(x.type(gpu_dtype))
#     y_var = Variable(y.type(gpu_dtype).long())

#     # This is the forward pass: predict the scores for each class, for each x in the batch.
#     scores = fixed_model_gpu(x_var)
    
#     # Use the correct y values and the predicted y values to compute the loss.
#     loss = loss_fn(scores, y_var)
    
#     if (t + 1) % print_every == 0:
#         print('t = %d, loss = %.4f' % (t + 1, loss.item()))

#     # Zero out all of the gradients for the variables which the optimizer will update.
#     optimizer.zero_grad()
    
#     # This is the backwards pass: compute the gradient of the loss with respect to each 
#     # parameter of the model.
#     loss.backward()
    
#     # Actually update the parameters of the model using the gradients computed by the backwards pass.
#     optimizer.step()

In [6]:
# Train your model here, and make sure the output of this cell is the accuracy of your best model on the 
# train, val, and test sets. Here's some code to get you started. The output of this cell should be the training
# and validation accuracy on your best model (measured by validation accuracy).

# model = nn.Sequential(
#             nn.Conv2d(3, 32, kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(32, 64, kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(2),
#             nn.Conv2d(64, 16, kernel_size=3),
#             nn.ReLU(),
#             nn.Conv2d(16, 32, kernel_size=3),
#             nn.ReLU(),
#             nn.MaxPool2d(2),
#             Flatten(),
#             nn.Linear(32*5**2, 1024),
#             nn.ReLU(),
#             nn.Linear(1024, 10)
#             ).type(gpu_dtype)

resnet50 = models.resnet50()#.cuda() # pretrained=True is pretrained on ImageNet

loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = optim.RMSprop(resnet50.parameters(), lr=1e-3)

In [7]:
import importlib
importlib.reload(utils)

<module 'utils' from '/home/braedon/skole/cs175/PlantPathology2021/utils.py'>

In [8]:
# test

# for x,y in loader_tr:
#     print(x,y)
#     break

# print(imgs_tr[0].shape)
# print(type(imgs_tr[0][0,0,0,0]))

# torch.from_numpy(X_tr).float().cuda()
# imgs_tr[1]

# type(imgs_tr[0].[0,0,0,0])
# torch.from_numpy(imgs_tr[0][0][np.newaxis,:]).float().cuda()

In [9]:
utils.train(resnet50, imgs_tr[0], imgs_tr[1], loss_fn, optimizer, num_epochs=50)
utils.check_accuracy(resnet50, loader_val)

Starting epoch 1 / 50
0


TypeError: 'int' object is not callable